In [2]:
import torch
from moviepy.editor import VideoFileClip
from librosa import feature

In [ ]:
class AudioExtractor:
    def __init__(self, video_path):
        self.video_clip = VideoFileClip(video_path)
        self.audio_clip = self.video_clip.audio

    def extract_features(self, data: any) -> torch.Tensor:
        mfccs = feature.mfcc(self.audio_clip) #MFCC — один из наиболее распространённых признаков, который широко используется в задачах распознавания речи и звука.
        mel_spectrogram = feature.melspectrogram(y=self.audio_clip, sr=self.audio_clip.fps, n_mels=128) #показывает распределение энергии в частотном диапазоне по времени
        
        
        
        
        
        
        
        
        return torch.tensor([1.0])